In [ ]:
# helpful munging scripts
# Callin Switzer
# 15 Jan 2016
# updated 17 Nov 2017


In [1]:
# import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time
import re
from datetime import datetime

In [ ]:
 %qtconsole

In [ ]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [ ]:
aa = file_choose()

In [ ]:
print aa

In [ ]:
# os.chdir('/Volumes/GMD/BeeSonicationLearningWithAvery/BeeFrequencyLearning/')
os.chdir('/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/CompressedBeeFrequencyLearning/')

In [ ]:
fldrs = [f for f in os.listdir(".") if not f.startswith('.')]
len(fldrs)

In [ ]:
fldrs.sort(key=lambda x: os.path.getmtime(x)) # sort by time created

In [ ]:
# fldrs.reverse()
fldrs

In [ ]:
import csv

In [ ]:
with open("/Users/cswitzer/Desktop/output.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(np.transpose(np.array([fldrs])))

In [ ]:
np.transpose(np.array([fldrs])).shape

In [ ]:
file_choose()

In [ ]:
# copy files and exclude movies
import shutil

In [ ]:
os.chdir('/Volumes/GMD/19 Dec Bee Sonication Backup/BeeSonicationLearningWithAvery/BeeFrequencyLearning/')

In [ ]:
sourceDir = os.getcwd()
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_NoImages'
IGNORE_PATTERNS = ('*.pgm')


In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
sourceDir = '/Volumes/GMD/BeeSonicationLearningWithAvery/HeavyLightFlowers/'
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_HeavyLight'
IGNORE_PATTERNS = ('*.pgm')


In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
# pick up here

In [ ]:
sourceDir = '/Volumes/GMD/BeeSonicationLearningWithAvery/TestAndTrainFakeAndRealFlowers/'
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_TestTrainDataset_noVid'
IGNORE_PATTERNS = ('*.pgm', '*.mp4')

In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
# done

In [ ]:
# START HERE

In [106]:
# os.chdir('/Users/callinswitzer/Desktop/SonicationLearningWAvery_NoImages/')

os.chdir('/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/SonicationLearningWAvery_NoImages/')

In [114]:
mypath = os.getcwd()
mypath

'/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/SonicationLearningWAvery_NoImages'

In [109]:
folders

['Beeredblue5_29Nov2016_Hive4_initial',
 'Beewhitepink9_06Dec2016_Hive5_high',
 'Beeyellowblue8_14Dec2016_Hive5_high',
 'Beelimeorange2_18Dec2016_Hive5_NoReward',
 'Beewhite5_30Nov2016_Hive4_initial',
 'Beeredpink10_14Dec2016_Hive5_low',
 'Beewhitered4_07Dec2016_Hive5_high',
 'Beeorange7_09Dec2016_Hive3_high',
 'Beeredblue9_30Nov2016_Hive4_initial',
 'Beelimepink_18Dec2016_Hive5_NoReward',
 'Beelimered1_14Dec2016_Hive5_initial',
 'Beeredblue6_30Nov2016_Hive4_initial',
 'Beeredblue11_02Dec2016_Hive4_initial',
 'Beeyellowgreen8_12Dec2016_Hive5_initial',
 'Beewhitegold3_06Dec2016_Hive5_low',
 'Beeredpink1_10Dec2016_Hive5_initial',
 'Beelime1_12Dec2016_Hive5_initial',
 'Beewhite6_02Dec2016_Hive4_initial',
 'Beeyellowpink10_14Dec2016_Hive5_low',
 'Beeyellowgreen2_07Dec2016_Hive5_initial',
 'Beewhiteyellow1_04Dec2016_Hive5_initial',
 'Beeyelloworange10_11Dec2016_Hive5_high',
 'Beelimeblue7_15Dec2016_Hive5_initial',
 'Beeyellowgreen6_10Dec2016_Hive5_initial',
 'BeeBlue6_28Nov2016_Hive4_high',

In [130]:
# for each folder, open it, and read the ampFreq.txt file

probs = 0
for ii in range(len(folders)):

    # read contents of each folder
    fcontents = [f for f in os.listdir(folders[ii]) if not f.startswith('.')]

    # get ampFreq file(s). there should be one per folder
    ampFreqFile = [x for x in fcontents if 'ampFreq.txt' in x]

    if len(ampFreqFile) > 1: 
        print("PROBLEM " + str(ii))


    # read ampFreq file
    tmpDF = pd.read_table(os.path.join(folders[ii], ampFreqFile[0]), header = None, sep = ',')

    # add folder name to data frame
    tmpDF[7] = folders[ii]

    # add frequency information
    folderInFolder = [x for x in fcontents if not 'ampFreq.txt' in x]

    if len(folderInFolder) > 1: 
        print("PROBLEM")
        probs += 1

    # list files in inner folder
    accRecFiles = [x for x in os.listdir(os.path.join(folders[ii], folderInFolder[0])) if not x.startswith('.')]
    accRecFiles.sort()

    tmpDF[8] = accRecFiles

    # remove wingbeats
    tmpDF2 = tmpDF.loc[(tmpDF[0] > 220) & (tmpDF[0] < 450)]

    # renumber index
    tmpDF2.index = (np.arange(1, tmpDF2.shape[0] + 1))

    # write to .csv
    tmpDF2.to_csv('/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/FreqTrainCSVs/' + folders[ii] + '.csv', 
                 header = False, index = True)
    
    if np.mod(ii, 10) == 0:
        print ii
    
print str(probs) + " problems"

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
0 problems


In [131]:
## combine csv's all into a single file
csvDir = '/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/FreqTrainCSVs'

csvFiles = [f for f in os.listdir(csvDir) if f.endswith('.csv')]

np_array_list = []
for file_ in csvFiles:
    df = pd.read_csv(os.path.join(csvDir, file_),index_col=None, header=None)
    np_array_list.append(df.as_matrix())

comb_np_array = np.vstack(np_array_list)
big_frame = pd.DataFrame(comb_np_array)

big_frame.columns = ['index', 'freq', 'amp', 'datetime', 'rewNum', 'rewTF', 'lowRewAmp', 'highrewAmp', 'BeeNumCol', 'accFile']

big_frame.head()

,index,freq,amp,datetime,rewNum,rewTF,lowRewAmp,highrewAmp,BeeNumCol,accFile
0,1,340,0.69799,2016_12_11__14_51_35_856,1,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_35_856_220_450_test.txt
1,2,290,0.31249,2016_12_11__14_51_37_400,2,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_37_400_220_450_test.txt
2,3,310,0.49291,2016_12_11__14_51_38_788,3,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_38_788_220_450_test.txt
3,4,310,0.55911,2016_12_11__14_51_39_641,4,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_39_641_220_450_test.txt
4,5,350,0.30275,2016_12_11__14_51_41_043,5,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_41_043_220_450_test.txt


In [132]:
# save big data frame
big_frame.to_csv('/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/freqLearn.csv', header = True, index = False)

In [133]:
plt.hist(big_frame.freq)
plt.plot(big_frame.freq, [0.01]*len(big_frame.freq), '|', color='k')

In [134]:
#big_frame = pd.read_csv('/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/freqLearn.csv', index_col=None)

In [135]:
big_frame.head()

,index,freq,amp,datetime,rewNum,rewTF,lowRewAmp,highrewAmp,BeeNumCol,accFile
0,1,340.0,0.69799,2016_12_11__14_51_35_856,1,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_35_856_220_450_test.txt
1,2,290.0,0.31249,2016_12_11__14_51_37_400,2,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_37_400_220_450_test.txt
2,3,310.0,0.49291,2016_12_11__14_51_38_788,3,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_38_788_220_450_test.txt
3,4,310.0,0.55911,2016_12_11__14_51_39_641,4,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_39_641_220_450_test.txt
4,5,350.0,0.30275,2016_12_11__14_51_41_043,5,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_41_043_220_450_test.txt


In [136]:
print big_frame.shape

(26552, 10)


In [138]:
# check to see if datetime and accFile agree

eqs = [big_frame['datetime'][ii][1:25] == big_frame['accFile'][ii][0:24] for ii in range(len(big_frame['datetime']))]

In [139]:
np.sum(np.invert(eqs)) # should be zero if all of them match

0

In [141]:
# find the one that doesn't match
big_frame.loc[np.invert(eqs)] # now they all match

,index,freq,amp,datetime,rewNum,rewTF,lowRewAmp,highrewAmp,BeeNumCol,accFile


In [142]:
# make a new column for bee color, hive, reward frequency, date, and treatment (initial, high, low)

str1 = big_frame['BeeNumCol'][1]

In [143]:
str1

'Beesilver1_11Dec2016_Hive5_initial'

In [144]:
str2 = [big_frame['BeeNumCol'][ii].split("Bee")[1] for ii in range(len(big_frame['BeeNumCol']))]

In [145]:
% qtconsole

In [146]:
beeColNum = [str2[ii].split(r'_')[0] for ii in range(len(str2))]

In [148]:
# definition for extracting trial number
def extractNum(string):
    try: 
        aa =  str(int(filter(str.isdigit, string)))
    except:
        aa = "1"
    return(aa)
    

In [149]:
# get trial number
trialNum = [extractNum(strg) for strg in beeColNum]

In [150]:
# function to get only characters
def extractChar(string):
    return(" ".join(re.findall("[a-zA-Z]+", string)))

In [151]:
# get just bee color
beeCol = [extractChar(strg) for strg in beeColNum]

In [152]:
# get datetime
dateTime = [datetime.strptime(dt1, ' %Y_%m_%d__%H_%M_%S_%f') for dt1 in big_frame['datetime']]

In [153]:
# string format time
dateTime_format = [datetime.strftime(datetime_object, "%Y-%m-%d %H:%M:%S.%f") for datetime_object in dateTime]

In [154]:
# get reward frequencies
s1 = big_frame['accFile'][0]

In [155]:
rewFrqs = [s1.split("_")[8:10] for s1 in big_frame['accFile']]

In [156]:
rewDF = pd.DataFrame(rewFrqs, columns = ['lowFrq', 'highFrq'])

In [157]:
rewDF.head()

,lowFrq,highFrq
0,220,450
1,220,450
2,220,450
3,220,450
4,220,450


In [158]:
# add to big data frame
big_frame['trialNum'] = trialNum
big_frame['beeCol'] = beeCol
big_frame['datetime_str'] = dateTime_format

In [159]:
big_frame2 = pd.concat([big_frame, rewDF], axis  = 1)
big_frame2.head()

,index,freq,amp,datetime,rewNum,rewTF,lowRewAmp,highrewAmp,BeeNumCol,accFile,trialNum,beeCol,datetime_str,lowFrq,highFrq
0,1,340.0,0.69799,2016_12_11__14_51_35_856,1,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_35_856_220_450_test.txt,1,silver,2016-12-11 14:51:35.856000,220,450
1,2,290.0,0.31249,2016_12_11__14_51_37_400,2,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_37_400_220_450_test.txt,1,silver,2016-12-11 14:51:37.400000,220,450
2,3,310.0,0.49291,2016_12_11__14_51_38_788,3,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_38_788_220_450_test.txt,1,silver,2016-12-11 14:51:38.788000,220,450
3,4,310.0,0.55911,2016_12_11__14_51_39_641,4,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_39_641_220_450_test.txt,1,silver,2016-12-11 14:51:39.641000,220,450
4,5,350.0,0.30275,2016_12_11__14_51_41_043,5,T,0,5,Beesilver1_11Dec2016_Hive5_initial,2016_12_11__14_51_41_043_220_450_test.txt,1,silver,2016-12-11 14:51:41.043000,220,450


In [160]:
#big_frame2.to_csv('/Users/cswitzer/Dropbox/SonicationLearningManuscript/Data/freqLearn2_updated.csv', header = True, index = False)